# IUM projekt - etap 2
### Jakub Grzechociński  
### Piotr Sawicki


## Treść zadania
> Są osoby, które wchodzą na naszą stronę i nie mogą się zdecydować, którym produktom przyjrzeć się nieco lepiej. Może dało by się im coś polecić?

## Streszczenie etapu pierwszego
Do stworzenia jest system oferujący usługę rekomendacji produktów dla klientów sklepu internetowego. System opiera swoje działanie na danych przejrzeń i zakupów produktów przez użytkowników sklepu oraz na danych samych produktów.

System oferuje rekomendacje na podstawie dwóch oddzielnych modeli:
- `popularnościowy` bazujący na względnej popularności produktów i kategorii oraz preferencji użytkownika
- `collaborative filtering` odtwarzający podobieństwa między klientami

W ramach implementacji systemu należy utworzyć interfejs dla modeli rekomendacji w postaci mikroserwisu. W wyniku jego działania aplikacje klienckie mogą uzyskiwać rekomendacje produktów na podstawie id użytkownika. System pozwala na działanie z wykorzystaniem jednego z dwóch modeli lub w trybie testu A/B dla porównania modeli.

W etapie pierwszym wykonana została analiza danych, w ramach której zidentyfikowane zostały problemy oraz oszacowano, że system jest w stanie skutecznie rekomendować na podstawie danych.

## Zaimplementowane modele

### Model popularnościowy
Model opiera swoje działanie na względnej popularności przedmiotów oraz ich kategorii. Zliczana jest ilość wyświetlneń oraz zakupów każdego z produktów, przy czym zakup ma mnożnik X9 (jest to jeden z hiperparametrów modelu). Produkty są sortowane zgodnie z tak oszacowaną popularnością wewnątrz swoich karegorii. 

## Pokaz działania 

Serwis został umieszczony na maszynie w Oracle Cloud pod publicznym adresem ip `130.61.188.211`. Można więc przetestować jego działanie z dowolnego miejsca. Serwis odpowiada na odpowiednio sformułowane żądanie HTTP.   

Np. Aby uzyskać otrzymać odpowiedź zawierającą zestaw dziesięciu rekomendacji dla klienta o id `120` wystarczy wysłać zapytanie HTTP GET na adres: 
``` 
 130.61.188.211/recommendations?user_id=121&num=10 
``` 
W linii poleceń można szybko wykonać takie zapytanie przy pomocy narzędzia `curl`: 
``` 
curl -i "130.61.188.211/recommendations?user_id=121&num=10" 
``` 
Odpowiedź serwera: 
``` 
HTTP/1.1 200 OK 
Content-Type: application/json 
Date: Sun, 28 Mar 2021 16:30:32 GMT 
Content-Length: 110 
  
{"type":"user","id":121,"attributes":{"recommendations":[1077,1076,1080,1095,1004,1005,1006,1007,1008,1009]}} 
``` 

Serwer chodzący pod publicznym IP pracuje w trybie testu A/B. Przyporządkowanie użytkowników do modelu zapisywane jest w bazie danych SQLite. Po stronie serwera możemy sprawdzić, który system wygenerował ten zestaw rekomendacji: 
``` 
sqlite> SELECT * FROM user_to_model WHERE user_id == 121; 
user_id|model_id 
121|2 
``` 
Następnie szybki wgląd w przypisanie `model_id` do zrozumiałej nazwy: 
``` 
sqlite> SELECT * FROM models; 
model_id|name|user_count 
1|Popularity|4 
2|Collaborative|4 
``` 

Dla zapytania dla użytkowinika o `user_id` 121 otrzymaliśmy więc rekomendacje od systemu `Collaborative`, czyli modelu CF. Na podstawie tak zbieranych danych oraz dalszych danych zbieranych przez sklep internetowy można oszacować skuteczność obu modeli. 

Warto by było teraz sprawdzić jakie produkty zostały zarekomendowane. Są to produkty o `product_id`: 
```
1077,1076,1080,1095,1004,1005,1006,1007,1008,1009 
``` 

In [130]:
import datetime
import numpy as np
import pandas as pd

data_dir = 'data/'
data_raw_dir = 'data_raw/'
products_file = 'products.jsonl'
sessions_file = 'sessions.jsonl'
products_filepath = data_raw_dir + products_file
sessions_filepath = data_dir + sessions_file
products_data = pd.read_json(products_filepath, convert_dates=False, lines=True)
products_data = products_data.drop(columns=['price'])
sessions_data = pd.read_json(sessions_filepath, convert_dates=False, lines=True)

In [134]:
product_list = [1283, 1001, 1318, 1277, 1276, 1278, 1281, 1035, 1234, 1233, 1067, 1317, 1315, 1072, 1316, 1222, 1201, 1319, 1073, 1074, 1047, 1043, 1040, 1084, 1045, 1046, 1075, 1002, 1081, 1003, 1077, 1079, 1076, 1044, 1041, 1078, 1034, 1080, 1032, 1069, 1306, 1064, 1030, 1017, 1037, 1039, 1025, 1033, 1038, 1070]
# get only the products from list
recommended_products = products_data[products_data.product_id.isin(product_list)]
# index them in the order from the list
recommended_products.reindex(recommended_products.product_id.map({x: i for i, x in enumerate(product_list)}).sort_values().index)

,product_id,product_name,category_path
282,1283,Okulary 3D PHILIPS PTA436/00,Sprzęt RTV;Video;Telewizory i akcesoria;Okular...
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne
317,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki
276,1277,Apple iPad mini 64GB 4G,Komputery;Tablety i akcesoria;Tablety
275,1276,Apple iPad mini 64GB,Komputery;Tablety i akcesoria;Tablety
277,1278,Intenso Music Walker 8GB,Sprzęt RTV;Przenośne audio i video;Odtwarzacze...
280,1281,Manta MM266,Sprzęt RTV;Przenośne audio i video;Odtwarzacze...
34,1035,Samsung Galaxy S III GT-i9300,Telefony i akcesoria;Telefony komórkowe
233,1234,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD
232,1233,Manta DVD064,Sprzęt RTV;Video;Odtwarzacze DVD


Produkty najlepsze zdaniem modelu są wyżej, gorsze niżej. Model zdaje się polecać klientowi przede wszystkim skanery.

In [132]:
target_user_id = 302
# Odrzucenie danych nie związanych z użytkownikiem o id 302
sessions_data = sessions_data.loc[sessions_data['user_id']==target_user_id]
# Złączenie danych sesji i produktów
session_join_products = sessions_data.join(products_data.set_index('product_id'), on='product_id', how='left')
session_join_products = session_join_products[['session_id', 'user_id', 'event_type', 'category_path', 'product_name']]
session_join_products

,session_id,user_id,event_type,category_path,product_name
83034,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,Fallout 3 (Xbox 360)
83035,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,Dead Space 3 (Xbox 360)
83036,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry PlayStation3,GTA 5 (PS3)
83037,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,GTA 4 (Xbox 360)
83038,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,Mass Effect 2 (Xbox 360)
83039,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,Fallout 3 (Xbox 360)
